In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite.metrics import flat_classification_report

In [2]:
#Reading the csv file
df = pd.read_csv('ner_dataset.csv')
df

,Sentence #,Word,POS,Tag
0,Sentence: 1,The,DT,O
1,NaN,wife,NN,REL
2,NaN,was,VBD,O
3,NaN,killed,VBD,VIO_V
4,NaN,by,IN,O
5,NaN,the,DT,O
6,NaN,husband,NN,Actor
7,Sentence: 2,The,DT,O
8,NaN,grandfather,NN,Actor
9,NaN,abused,VBD,VIO_V


In [3]:
df.head(10)

,Sentence #,Word,POS,Tag
0,Sentence: 1,The,DT,O
1,NaN,wife,NN,REL
2,NaN,was,VBD,O
3,NaN,killed,VBD,VIO_V
4,NaN,by,IN,O
5,NaN,the,DT,O
6,NaN,husband,NN,Actor
7,Sentence: 2,The,DT,O
8,NaN,grandfather,NN,Actor
9,NaN,abused,VBD,VIO_V


In [4]:
df.describe()

,Sentence #,Word,POS,Tag
count,3,16,16,16
unique,3,15,5,4
top,Sentence: 2,The,NN,O
freq,1,2,6,7


In [5]:
#Displaying the unique Tags
df['Tag'].unique()

array(['O', 'REL', 'VIO_V', 'Actor'], dtype=object)

In [6]:
#Checking null values, if any.
df.isnull().sum()

Sentence #    13
Word           0
POS            0
Tag            0
dtype: int64

In [7]:
df = df.fillna(method = 'ffill')

In [8]:
# This is a class to get sentence. Each sentence will be a list of tuples with its tag and POS.
class sentence(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(), s['POS'].values.tolist(), s['Tag'].values.tolist())]
        self.grouped = self.df.groupby("Sentence #").apply(agg)
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None


In [9]:
#Displaying one full sentence
getter = sentence(df)
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

'The wife was  killed by  the  husband'

In [10]:

#sentence with its pos and tag.
sent = getter.get_text()
print(sent)

[('The', 'DT', 'O'), ('wife', 'NN', 'REL'), ('was ', 'VBD', 'O'), ('killed', 'VBD', 'VIO_V'), ('by ', 'IN', 'O'), ('the ', 'DT', 'O'), ('husband', 'NN', 'Actor')]


In [11]:
sentences = getter.sentences



In [12]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [13]:

X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
y


[['O', 'REL', 'O', 'VIO_V', 'O', 'O', 'Actor'],
 ['O', 'Actor', 'VIO_V', 'O', 'REL'],
 ['O', 'Actor', 'VIO_V', 'REL']]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
X_test


[[{'bias': 1.0,
   'word.lower()': 'the',
   'word[-3:]': 'The',
   'word[-2:]': 'he',
   'word.isupper()': False,
   'word.istitle()': True,
   'word.isdigit()': False,
   'postag': 'DT',
   'postag[:2]': 'DT',
   'BOS': True,
   '+1:word.lower()': 'wife',
   '+1:word.istitle()': False,
   '+1:word.isupper()': False,
   '+1:postag': 'NN',
   '+1:postag[:2]': 'NN'},
  {'bias': 1.0,
   'word.lower()': 'wife',
   'word[-3:]': 'ife',
   'word[-2:]': 'fe',
   'word.isupper()': False,
   'word.istitle()': False,
   'word.isdigit()': False,
   'postag': 'NN',
   'postag[:2]': 'NN',
   '-1:word.lower()': 'the',
   '-1:word.istitle()': True,
   '-1:word.isupper()': False,
   '-1:postag': 'DT',
   '-1:postag[:2]': 'DT',
   '+1:word.lower()': 'was ',
   '+1:word.istitle()': False,
   '+1:word.isupper()': False,
   '+1:postag': 'VBD',
   '+1:postag[:2]': 'VB'},
  {'bias': 1.0,
   'word.lower()': 'was ',
   'word[-3:]': 'as ',
   'word[-2:]': 's ',
   'word.isupper()': False,
   'word.istitle()': 

In [15]:
crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         max_iterations = 100,
         all_possible_transitions = False)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', all_possible_transitions=False, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [16]:
#Predicting on the test set.
y_pred = crf.predict(X_test)

In [17]:
print(y_pred)


[['O', 'Actor', 'VIO_V', 'VIO_V', 'O', 'O', 'REL']]
